In [1]:
from simtk import openmm, unit
from simtk.openmm import app
from openmmtools.testsystems import WaterBox

# Running SaltSwap
Brief notebook on how to use `saltswap`. All examples will use a small box of water, created below.

In [2]:
 # Setup box of water
size = 25.0*unit.angstrom     # The length of the edges of the water box.
temperature = 300*unit.kelvin
pressure = 1*unit.atmospheres
wbox = WaterBox(box_edge=size,nonbondedMethod=app.PME,cutoff=9*unit.angstrom,ewaldErrorTolerance=1E-5)

## 1. Using the `MCMCSampler` wrapper
The simplest way to get allow the salt concentration to vary in a given simulation. `MCMCSampler` is class that wraps molecular dynamics with `openmm` and Monte Carlo salt fluctuations from `saltswap` into a single object. 

* The MCMCSampler.move(nmove) function alternates between molecular dynamics and salt fluctuation moves for nmove iterations

In [3]:
from saltswap.mcmc_samplers import MCMCSampler

### Set-up and run simulations

#### Simulation variables

In [4]:
# NCMC parameters
npert = 100                       # The number of perturbation steps
nprop = 1                          # The number of propagation steps per NCMC perturbation
timestep = 1.0*unit.femtoseconds   # The timestep for the NCMC propagator

# If nprop = 0, water and salt are instantaneously exchanged.

# Specifying the frequency of molecular dynamics and Monte Carlo salt fluctuations.
steps = 5000     # The number of consecutive molecular dynamics steps at a fixed salt concentration per iteration
attempts = 10   # The number salt insertion/deletion attempts per iteration

delta_chem = 300 # The chemical potential in multipes of thermal energy

platform = 'CPU'   # The type of platform to run the dynamics with. 
# Can be either 'CPU', 'CUDA', or 'OpenCL'

#### Create the object to simulate

In [5]:
sampler = MCMCSampler(wbox.system, wbox.topology, wbox.positions, temperature=temperature, pressure=pressure,
                      npert=npert, nprop=nprop, propagator='GHMC', ncmc_timestep = timestep,
                      delta_chem=delta_chem, mdsteps=steps, saltsteps=attempts, platform=platform)

#### Run simulation

In [6]:
# Equilibration of configuration
sampler.gen_config(mdsteps=1000)

# Alternate between molecular dynamics and Monte Carlo for 100 iterations
sampler.move(100)

### Viewing the results

In [7]:
# To view the number of salt molecules after the moves:
n_waters, n_cations, n_anions = sampler.saltswap.get_identity_counts()

# View the acceptance rate for the salt insertion/deletion moves:
acceptance_rate = sampler.saltswap.get_acceptance_probability()

# The lists of the work to add and remove salt for every insertion/deletion attempt:
work_to_add_salt = sampler.saltswap.work_add
work_to_remove_salt = sampler.saltswap.work_rm

## 2. Using `SaltSwap` directly
This grants you, the user, more control over the parameters of the simulation, but is more involved to set-up.
* SaltSwap is the name of the Monte Carlo driver for exchanging water molecules with anion and cation pairs. 

In [8]:
from saltswap.swapper import Swapper
from saltswap.integrators import GHMCIntegrator

### Set-up and run simulations

#### Simulation variables

In [9]:
# NCMC parameters
npert = 10     # The number of perturbation steps
nprop = 1        # The number of propagation steps per NCMC perturbation

# If nprop = 0, water and salt are instantaneously exchanged.

delta_chem = 300*unit.kilojoule_per_mole # The chemical potential in multipes of kT

platform = 'CPU'   # The type of platform to run the dynamics with

#### Create the object to simulate

In [10]:
# Create a compound inegrator for regular dynamics and NCMC moves. If the salt insertion/deletion 
# moves are instantaneous, a compound integrator is not required.
integrator = openmm.CompoundIntegrator()
# 1. The integrator for regular molecular dynamics
integrator.addIntegrator(openmm.LangevinIntegrator(temperature, 1/unit.picosecond, 2.0*unit.femtoseconds))
# 2. The integrator for NCMC
integrator.addIntegrator(GHMCIntegrator(temperature, 1/unit.picosecond, 1.0*unit.femtoseconds, nsteps=nprop))

# Create the context
context = openmm.Context(wbox.system, integrator)
context.setPositions(wbox.positions)
context.setVelocitiesToTemperature(temperature)

# Create the object to swap salt with water
swapper =  Swapper(system=wbox.system, topology=wbox.topology,temperature=temperature, delta_chem=delta_chem,
                    integrator=integrator, pressure=pressure, npert=npert, nprop=nprop)

#### Run the simulation
The simulation alternates between molecular dynamics and `saltswap` moves. Below runs a *very* short example just to demonstrate functionality.

In [11]:
for iteration in range(10):
    integrator.step(100)                      # Run molecular dynamics for 1000 steps
    swapper.update(context, nattempts=1)      # Attempt 10 insertions or deletions

/Users/rossg/miniconda2/lib/python2.7/site-packages/saltswap-0.1.dev0-py2.7.egg/saltswap/swapper.py:703: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  molecule1 = [atom for atom in self.mutable_residues[exchange_indices[0]].atoms()]
/Users/rossg/miniconda2/lib/python2.7/site-packages/saltswap-0.1.dev0-py2.7.egg/saltswap/swapper.py:704: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  molecule2 = [atom for atom in self.mutable_residues[exchange_indices[1]].atoms()]


It's also possible to constrain the maximum amount of salt in a simulation with the `saltmax` variable.

In [12]:
for iteration in range(10):
    integrator.step(100)                                  # Run molecular dynamics for 1000 steps
    swapper.update(context, nattempts=1, saltmax=10)      # No more than 'saltmax' pairs can be inserted

### Viewing the results

In [13]:
# To view the number of salt molecules after the moves:
n_waters, n_cations, n_anions = swapper.get_identity_counts()

# View the acceptance rate for the salt insertion/deletion moves:
acceptance_rate = swapper.get_acceptance_probability()

# The lists of the work to add and remove salt for every insertion/deletion attempt:
work_to_add_salt = swapper.work_add
work_to_remove_salt = swapper.work_rm